# Outline 

 - Dataflow Programming
     - Intro to Dataflow Programming
     - Operations and Tensors


 - The Graph is defined as a protobuf
 	- Exploring the graph protobuf
 	- Exporting and editing the graph protobuf


 - How to execute a graph program
 	- Executing our first session
 	- Special Operation Placeholders


 - Graph building conventions
 	- Opinionated convention
 	- All about naming
 	- operation and tensor naming convention
 	- get_tensor and get_operation




# Dataflow Programming

###  Intro to Dataflow Programming
Before jumping into TensorFlow, we should understand the general paradigm of [Dataflow or Datastream programming](https://en.wikipedia.org/wiki/Dataflow_programming). At the end of this section, you should be able to use TensorFlow as an overly complicated scientific calculator. 

The main idea behind Dataflow is to define computation graphs. Each node in the graph represents a function such as addition, matrix multiplication, etc. Each edge represents the inputs/outputs of these functions. Once the computation graph is defined, we can evaluate nodes of interest.

<img src="../figures/graph.png" width="400"/>

In the example above, we would like to evaluate $G$. You may notice a couple things:

1. Only a subgraph, which excludes node $F$, is required to evaluate node $G$ outputs. 

2. Collections of contiguous nodes can conceptually behave as a single node. For example, the collection of $\{C, D, E\}$ takes one input from $\{B\}$ and produces two outputs leading towards $\{F\}$ and $\{G\}$.


We describe some very desirable properties: 
1. Only executing necessary subgraphs means lower computation overhead.

2. Being able to form contiguous node subgraphs allow for distributed function placement across multiple devices. 

These are just [some](https://www.tensorflow.org/programmers_guide/graphs#why_dataflow_graphs) of the design reasons TensorFlow follows the Dataflow programming model.

So enough theory, how does TensorFlow implement the Dataflow model? TensorFlow implements the Dataflow execution model in C++ known as the "TensorFlow runtime". Interfacing with the TensorFlow runtime are low level APIs currently available in [Python](https://www.tensorflow.org/api_docs/python), [GoLang](https://godoc.org/github.com/tensorflow/tensorflow/tensorflow/go), [Java](https://www.tensorflow.org/api_docs/java/reference/org/tensorflow/package-summary), and [C++](https://www.tensorflow.org/api_docs/cc). These low level APIs build *Graphs*, and execute the TensorFlow runtime in the form of *Sessions*. Computation graphs are ultimately executed on device hardware (CPU, GPU, [TPU](https://en.wikipedia.org/wiki/Tensor_processing_unit)). We will be focusing on the Python API to create Graphs, and run Sessions.

<img src="../figures/API_layers.png" width="400"/>

### Operations and Tensors

In TensorFlow’s computation graph, nodes are [Operations](https://www.tensorflow.org/versions/master/api_docs/python/tf/Operation), and edges are [Tensors](https://www.tensorflow.org/versions/master/api_docs/python/tf/Tensor). Operations then simply have input and output Tensors. Let's recreate our previous example graph with arbitrary ops in tensorflow. These basic arbitrary ops can be found [here](https://www.tensorflow.org/api_docs/python/tf#functions). The graph can be visualized here via iframe ([thanks jakub arnold!](https://blog.jakuba.net/2017/05/30/tensorflow-visualization.html)), or visualized on your local tensorboard at `localhost:6006`

We create a graph `g1` and add operations $\{A,B,C,D,E,F,G\}$. Note that `tf.constant` is considered an operation, it takes in no tensors as input and produces a single tensor as output. It is a subtle, but important distiction to make that all nodes in the TensorFlow graph are  [Operations](https://www.tensorflow.org/versions/master/api_docs/python/tf/Operation).

In the python API, `tf.Operation` will return a `tf.Tensor` which can be wired as inputs into another `tf.Operation`. `tf.Tensor` was built analogously to [numpy arrays](https://docs.scipy.org/doc/numpy/reference/generated/numpy.array.html). Tensors have a [datatype](https://www.tensorflow.org/api_docs/python/tf/DType) such as `float32`, `int32`, or `string`. They can have a shape such as a scalar, vector, matrix, cube, with a corresponding dimension 0, 1, 2, 3. In this first example, we will just deal with `float32` scalar tensors to keep it simple.


If you run the code above you should successfully build a TensorFlow graph. Congrats!

In [14]:
import tensorflow as tf
from src import cloud_visualizer

g1 = tf.Graph() 

with g1.as_default(): 
    a = tf.constant(-0.8, name="A") # or a = -0.8
    b = tf.abs(a, name="B")
    
    c = tf.cos(b, name="C")
    d = tf.ceil(b, name="D")
    
    e = tf.multiply(c, d, name="E")
    f = tf.acos(c, name="F")
    
    g = tf.asin(e, name="G")
    
tf.summary.FileWriter("logs", g1).close() # write graph out to tensorboard for visualization
cloud_visualizer.show_graph(g1)

# The Graph is defined as a protobuf

### Exploring the graph protobuf

It is easy to get overwhelmed with all functions and operations related to `tf.Graph`. In the end, just remember that all we have done so far is defining some Dataflow structure. To ingrain this into our mind, I find it helpful to understand how `tf.Graph` is serialized or represented under the hood.

`tf.Graph` can be represented as a [protocol buffer](https://developers.google.com/protocol-buffers/docs/overview) known as [`tf.GraphDef`](https://www.tensorflow.org/api_docs/python/tf/GraphDef).

If you have worked with data formats such as `json` or `xml`, `protobufs` are just another structure format created by Google. TensorFlow uses a number of `protobuf` definitions: [`GraphDef`](https://github.com/tensorflow/tensorflow/blob/r1.3/tensorflow/core/framework/graph.proto), [`NodeDef`](https://github.com/tensorflow/tensorflow/blob/r1.3/tensorflow/core/framework/node_def.proto), and [`AttrValue`](https://github.com/tensorflow/tensorflow/blob/r1.3/tensorflow/core/framework/attr_value.proto) to name a few. 

We can output the `GraphDef` of `g1` quickly by using the `tf.Graph.as_graph_def()` function to look at a its text form as a `.pbtxt` file.

In [17]:
from google.protobuf import text_format

# Let's export the graphdef protobuf as a readable text file.
with open("./storage/g1.pbtxt", "w") as f:
    f.write(text_format.MessageToString(g1.as_graph_def()))

g1.as_graph_def() # display the graph inline.

node {
  name: "A"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
        }
        float_val: -0.800000011921
      }
    }
  }
}
node {
  name: "B"
  op: "Abs"
  input: "A"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "C"
  op: "Cos"
  input: "B"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "D"
  op: "Ceil"
  input: "B"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "E"
  op: "Mul"
  input: "C"
  input: "D"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "F"
  op: "Acos"
  input: "C"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "G"
  op: "Asin"
  input: "E"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
versions {
  producer: 24
}

Looking at the `.pbtxt` files we can see that the `GraphDef` contains many `NodeDef` objects. Each of these has a `name`, `op`, `input`, and `attr`. 

 - `name` refers to the the name of operation, and we can grab specific ops using the [`tf.Graph.get_operation_by_name`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_operation_by_name) method. 

 - `op` refers to the TensorFlow `Operation` the node references. 

 - `input` refers to the outputs coming from other defined nodes, node outputs are then implicit.

 - `attr` is a map of attributes that vary depending on the specific to the `op` the node represents.

### Exporting and editing the graph protobuf

The definition laid out in `GraphDef` begs the question, can we build graphs without using the TensorFlow python API? The answer is yes. 

Let's output a graph as a protobuf text, edit it and import it back into python. In the future we obviously would want to use the python API for building graphs. We will be creating operation $Z$, which adds the outputs from operation $G$ and $F$.

However, I hope that going through the protobuf text files show that `tf.Graph` is simply an API to declare graph structure, and that we can even bypass the API completely by writing it as a `.pbtxt` file. To re-emphasize, **all we have done is declare a graph structure**, we have not executed anything. Operation $Z$ has been added to `g2.pbtxt` for your convenience. As you can see after running the code $Z$ on the graph.

In [20]:
from google.protobuf import text_format

# Let's import our new graphdef protobuf
with open("./storage/g2.pbtxt", "r") as f: 
    graphdef = text_format.Parse(f.read(), tf.GraphDef())

g2 = tf.Graph() 
with g2.as_default():
    tf.import_graph_def(graph_def=graphdef, name="") # import the graph def into our new graph.

cloud_visualizer.show_graph(g2)

# Execute a Graph through a Session

### Executing our first session

Now that we have the basics of constructing graphs, we want to execute to get the actual output. We do this in the form of `tf.Session` which creates an environment, grabs computation power, and places graph operations on their proper devices. Don't be suprised if your computer fans start turning. In our simple case, it defaults to the CPU.

We will pass in the graph we just built, `g2` to compute the tensor outputs of $Z$. By default, the name of this tensor is `Z:0`. We will go into these naming conventions later

In [23]:
sess = tf.Session(graph=g2);
print(sess.run(g2.get_tensor_by_name("Z:0")))

1.5708


### Special Operation Placeholders

With our current graph, we can only evaluate our nodes based on the constant we've defined. It would be nice to input data at the session runtime. This is what the special operation `tf.Placeholder` allows us to do.

We specify a datatype, and shape for the `tf.Placeholder` operation and we can pass in inputs using the `feed_dict` kwarg for a session. Let's rebuild the graph and try this. If you run the code you can see we are able to feed in several values of `a` to evaluate `Z:0`, clearly a useful feature.

In [39]:
import tensorflow as tf
from src import cloud_visualizer

g3 = tf.Graph() 

with g3.as_default(): 
    a = tf.placeholder(dtype=tf.float32, shape=None, name="A") # or a = -0.8
    b = tf.abs(a, name="B")
    
    c = tf.cos(b, name="C")
    d = tf.ceil(b, name="D")
    
    e = tf.multiply(c, d, name="E")
    f = tf.acos(c, name="F")
    
    g = tf.asin(e, name="G")
    
    z = tf.add(f, g, name="Z")
    
tf.summary.FileWriter("logs", g3).close() # write graph out to tensorboard for visualization
cloud_visualizer.show_graph(g3)

In [40]:
sess = tf.Session(graph=g3)
sess.run("Z:0", feed_dict={a: [[1, 2, -3], [2, 1, 0]]}) # Can take in arbitrary shapes of data to eval.

array([[ 1.57079625,  1.0167675 ,         nan],
       [ 1.0167675 ,  1.57079625,  0.        ]], dtype=float32)

# Graph naming conventions

### Opinionated convention

TensorFlow is not an opionionated API. It is very forgiving and there are many ways to build the same functional graph. For example, we can build `g1` like the following. Why do `a` and `e` still work if TensorFlow only works with Operations? Also what is the default name of an operation if I don't pass the name?

 - For `a`, TensorFlow will automatically convert "[tensor-like-objects](https://www.tensorflow.org/programmers_guide/graphs#tensor-like_objects)" into tf.constant operations returning the converted tensor. This makes `a` an Operation. 
 
 - For `e`, TensorFlow will map [appropiate tensorflow functions via python operator overloading methods](https://stackoverflow.com/questions/37900780/in-tensorflow-what-is-the-difference-between-tf-add-and-operator), turning `+`, `/`, `-`, and `*`, into, you guessed it, Operations.

With all the ways to create the same graph, we suggest a couple conventions to keep graphs 

In [41]:
import tensorflow as tf
from src import cloud_visualizer

g1 = tf.Graph() 

with g1.as_default(): 
    a = -0.8
    b = tf.abs(a)
    
    c = tf.cos(b)
    d = tf.ceil(b)
    
    e = c*d
    f = tf.acos(c)
    
    g = tf.asin(e)
    
tf.summary.FileWriter("logs", g1).close() # write graph out to tensorboard for visualization
cloud_visualizer.show_graph(g1)